In [ ]:
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_curve
from keras.utils import to_categorical
import tensorflow as tf
from keras_preprocessing.image import ImageDataGenerator
import keras
from keras import datasets, layers, models
import pandas as pd
from keras.models import Sequential, load_model
from sklearn.metrics import roc_curve, auc



In [ ]:
dataset_path = 'path to the dataset'

In [ ]:
image_size = (32,32)
batch_size = 10

#Data generators
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_batches = train_datagen.flow_from_directory(
    dataset_path,
    target_size=image_size,
    batch_size=batch_size,
    color_mode = 'grayscale',
    class_mode='sparse',
    subset='training',
    shuffle='True'
)

validation_batches = train_datagen.flow_from_directory(
    dataset_path,
    target_size=image_size,
    batch_size=batch_size,
    color_mode = 'grayscale',
    class_mode='sparse',
    subset='validation',
    shuffle='True'
)

test_batches = train_datagen.flow_from_directory(
    dataset_path,
    target_size=image_size,
    batch_size=batch_size,
    color_mode = 'grayscale',
    class_mode='sparse',
    shuffle=False,
    subset='validation',
)

In [ ]:
model = load_model(os.path.join('path to Model folder', 'best_val_loss.hdf5'))

In [ ]:
# Make predictions on the test set using the trained model
y_pred = model.predict(test_batches)

# Convert the predicted probabilities into binary class labels
y_pred_class = np.argmax(y_pred, axis=1)

# Convert the true labels into a one-hot encoded format
y_test = to_categorical(test_batches.classes)

# Define the number of classes
num_classes = y_pred.shape[1]



In [ ]:
print(num_classes)

In [ ]:
# Plot the precision-recall curve for each class
plt.rcParams['savefig.facecolor']='white'
plt.figure(figsize=(15,19))
for i in range(num_classes):
    precision, recall, thresholds = precision_recall_curve(y_test[:, i], y_pred[:, i])
    plt.plot(recall, precision, label='Class {}'.format(i))
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curves for main character model2')
plt.legend()
plt.savefig('P-RC MC M2.png', bbox_inches='tight')
plt.show()

In [ ]:
y_true = test_batches.classes
y_pred = model.predict(test_batches)
y_pred = tf.argmax(y_pred, axis = 1).numpy()

In [ ]:
fpr = dict()
tpr = dict()
roc_auc = dict()

for i in range(19):
    y_true_binary = np.zeros(y_true.shape)
    y_true_binary[np.where(y_true==i)] = 1
    
    y_pred_binary = np.zeros(y_pred.shape)
    y_pred_binary[np.where(y_pred==i)] = 1
    
    fpr[i], tpr[i], _ = roc_curve(y_true_binary, y_pred_binary)
    roc_auc[i] = auc(fpr[i], tpr[i])

plt.rcParams['savefig.facecolor']='white'   
plt.figure(figsize=(10,9))
for i in range(19):
    plt.plot(fpr[i], tpr[i], label='Class {0} (AUC = {1:0.2f})'.format(i, roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curves for One-vs-All Binary Classification \n for half character model1')
plt.legend(loc="lower right")
plt.savefig('ROC for HC M1.png', bbox_inches='tight')
plt.show()